In [1]:
# 필요한 라이브러리 호출

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
import seaborn as sns # 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime

# 파일 위치 확인 (맥북 파일경로 복사 : command + option + c)
import os
print(os.listdir("/Users/bumchanpark/Desktop/MachineLearning_Study/AiffelResearch/data"))

['sampleSubmission.csv', 'test.csv', 'train.csv']


In [2]:
# 1) 훈련, 테스트 데이터셋의 개괄적인 형태 및 데이터의 컬럼의 속성 및 값의 개수 파악

# train, test 데이터셋을 불러온다
train = pd.read_csv('/Users/bumchanpark/Desktop/MachineLearning_Study/AiffelResearch/data/train.csv')
test = pd.read_csv('/Users/bumchanpark/Desktop/MachineLearning_Study/AiffelResearch/data/test.csv')

In [3]:
# .head() : train 데이터의 가장 앞의 5개의 데이터를 불러온다
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
"""
datetime - 시간별 날짜 + 타임스탬프
season - 1 = 봄, 2 = 여름, 3 = 가을, 4 = 겨울
holiday - 해당 요일이 휴일인지 여부
workingday - 해당 요일이 주말이나 휴일이 아닌지 여부
weather - 1: 맑음, 구름 조금, 구름 조금, 구름 조금
2: 안개 + 흐림, 안개 + 구름 조금, 안개 + 구름 조금, 안개
3: 가벼운 눈, 가벼운 비 + 뇌우 + 흩어진 구름, 가벼운 비 + 흩어진 구름
4: 폭우 + 유빙판 + 뇌우 + 안개, 눈 + 안개
temp - 섭씨 온도
atemp - 체감 온도(섭씨)
humidity - 상대 습도
windspeed - 풍속
casual - 미등록 사용자 대여 시작 횟수
registered - 등록된 사용자 대여 시작 횟수
count - 총 대여 횟수
"""

# 훈련 데이터셋의 각 컬럼별 데이터타입 및 값의 갯수 파악
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [5]:
# test 데이터의 가장 앞의 5개의 데이터를 불러온다
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [6]:
# 2) 데이터 전처리 및 시각화

# datetime 속성을 분리하여 추출 속성으로 활용하기 위해 split함수를 사용하여 년-월-일 과 시간을 분리한다
train['tempDate'] = train.datetime.apply(lambda x:x.split())

In [7]:
# 분리한 tempDate를 가지고 년-월-일을 이용하여 year,month,day 그리고 weekday column을 추출한다.
# split() :

train['year'] = train.tempDate.apply(lambda x:x[0].split('-')[0])
train['month'] = train.tempDate.apply(lambda x:x[0].split('-')[1])
train['day'] = train.tempDate.apply(lambda x:x[0].split('-')[2])

train['weekday'] = train.tempDate.apply(lambda x:calendar.day_name[datetime.strptime(x[0],"%Y-%m-%d").weekday()])
train['hour'] = train.tempDate.apply(lambda x:x[1].split(':')[0])

In [8]:
# 분리를 통해 추출된 속성은 문자열 속성을 가지고 있음 따라서 숫자형 데이터로 변환해 줄 필요가 있음

train['year'] = pd.to_numeric(train.year,errors='coerce')
train['month'] = pd.to_numeric(train.month,errors='coerce')
train['day'] = pd.to_numeric(train.day,errors='coerce')
train['hour'] = pd.to_numeric(train.hour,errors='coerce')

In [9]:
# year,month,day,hour가 숫자형으로 변환되었음을 알 수 있음
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
 12  tempDate    10886 non-null  object 
 13  year        10886 non-null  int64  
 14  month       10886 non-null  int64  
 15  day         10886 non-null  int64  
 16  weekday     10886 non-null  object 
 17  hour        10886 non-null  int64  
dtypes: float64(3), int64(12), object(3)
memory usage: 1.5+ MB


In [10]:
# 필요를 다한 tempDate column을 drop함
train = train.drop('tempDate',axis=1)